In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import vgg16
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
import os
import csv

In [ ]:
class FER2013(Dataset):
    def __init__(self, root, split="train", transform=None):
        assert split in ["train", "test"], "split must be either 'train' or 'test'"

        csv_file = f"{root}/{split}.csv"

        with open(csv_file, "r", newline="") as file:
            self.samples = [
                (
                    torch.tensor(
                        [int(idx) for idx in row["pixels"].split()], dtype=torch.uint8
                    ).reshape(48, 48),
                    int(row["emotion"]) if "emotion" in row else None,
                )
                for row in csv.DictReader(file)
            ]

        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_tensor, target = self.samples[idx]

        image_tensor.unsqueeze(0).expand(3, -1, -1)

        image = Image.fromarray(image_tensor.numpy())

        if self.transform is not None:
            image = self.transform(image)

        return image, target

In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

In [ ]:
# Define datasets and dataloaders
train_dataset = FER2013(
    root="./data/fer2013", split="train", transform=transform
)

valid_dataset = FER2013(
    root="./data/fer2013", split="test", transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the VGG16-based model
class CustomVGG16(nn.Module):
    def __init__(self, num_classes=7, pretrained=True):
        super(CustomVGG16, self).__init__()
        self.vgg16 = vgg16(pretrained=pretrained)
        self.vgg16.features[0] = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Flatten(),
            nn.BatchNorm1d(1000),
            nn.Linear(1000, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.vgg16(x)
        x = self.classifier(x)
        return x

In [ ]:
# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomVGG16()
model.to(device)
print(model)

# Loss function, optimizer, and metrics
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5, min_lr=1e-6)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomVGG16(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (1

In [ ]:
from torchsummary import summary

In [ ]:
summary(model, (3, 48, 48))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           1,792
              ReLU-2           [-1, 64, 48, 48]               0
            Conv2d-3           [-1, 64, 48, 48]          36,928
              ReLU-4           [-1, 64, 48, 48]               0
         MaxPool2d-5           [-1, 64, 24, 24]               0
            Conv2d-6          [-1, 128, 24, 24]          73,856
              ReLU-7          [-1, 128, 24, 24]               0
            Conv2d-8          [-1, 128, 24, 24]         147,584
              ReLU-9          [-1, 128, 24, 24]               0
        MaxPool2d-10          [-1, 128, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         295,168
             ReLU-12          [-1, 256, 12, 12]               0
           Conv2d-13          [-1, 256, 12, 12]         590,080
             ReLU-14          [-1, 256,

In [ ]:
# Training loop
epochs = 60

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Counter for training set
    train_counter = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        train_counter += labels.size(0)
        print(f'Training - Epoch [{epoch + 1}/{epochs}], Images Analyzed: {train_counter}/{len(train_dataset)}', end='\r')

    average_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_samples

    print(f'Training - Epoch [{epoch + 1}/{epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}, Images Analyzed: {train_counter}/{len(train_dataset)}')

    model.eval()
    val_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Counter for validation set
    valid_counter = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            valid_counter += labels.size(0)
            print(f'Validation - Epoch [{epoch + 1}/{epochs}], Images Analyzed: {valid_counter}/{len(valid_dataset)}', end='\r')

    val_loss /= len(valid_loader)
    accuracy = total_correct / total_samples

    print(f'Validation - Epoch [{epoch + 1}/{epochs}], Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}, Images Analyzed: {valid_counter}/{len(valid_dataset)}')

    scheduler.step(val_loss)

In [ ]:
# Save the model after training
torch.save({
    'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_loss': average_loss,
    'train_accuracy': accuracy,
}, 'saved_model.pth')

In [ ]:
# Evaluation
state = torch.load("./checkpoints/final_checkpoint_Horia.pth")
model.load_state_dict(state["model_state_dict"])
model.eval()
y_true, y_pred = [], []

accuracy = 0.0

import time

start_eval = time.time()
with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        expanded_inputs = inputs.expand(-1, 3, -1, -1)

        outputs = model(expanded_inputs)
        _, predicted = torch.max(outputs, 1)

        accuracy += (predicted == labels).sum()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
end_eval = time.time()

accuracy /= len(valid_dataset)
print(f"Accuracy is {accuracy.item() * 100}%")
print(f"Evaluation took {end_eval - start_eval:.2f}s")

# Print metrics
print("Evaluation Metrics:")
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Accuracy is 28.294789791107178%
Evaluation took 3.37s
Evaluation Metrics:
Confusion Matrix:
[[  75    0   96  130  260  342   55]
 [  11    0    5   17   30   43    5]
 [  75    0  124  101  239  350  135]
 [  40    0   31 1457   91  137   18]
 [  33    0   29   85  332  754   14]
 [  29    0   86   71   58   26  561]
 [  33    0   31   99  743  310   17]]
